# A Hubble Source Catalog (HSC) Use Case

- [Example #4: Using the Discovery Portal to perform cross-matching between an input catalog and the HSC][1]
  - (Search for HST data relevant to Supernova 2005cs in the galaxy M51)
  
![crossmatch_0][2]


  [1]: https://archive.stsci.edu/hst/hsc/help/use_case_4_v1.html
  [2]: screenshots/crossmatch_0.png

<span style="color:red;">Goal</span>: This tutorial shows you how to use the [MAST Discovery Portal][1] to perform cross-matching between an input catalog and the HSC. It also shows how to use the [Hubble Legacy Archive][2] to search for Hubble data that is not in the HSC.

<span style="color:red;">SCIENCE CASE</span>: The science case is to search for HST data relevant to a supernova (i.e. SN2005cs in the galaxy M51; see Maund et al. 2005, MNRAS, 364, 33, and Li et al. 2006, ApJ, 641, 1060).


  [1]: https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
  [2]: https://hla.stsci.edu/

In [ ]:
import time
import sys
import os
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from scipy.stats import linregress
from sklearn.linear_model import LinearRegression

from PIL import Image
from io import BytesIO, StringIO

from astroquery.mast import Observations
import astropy
from astropy.coordinates import SkyCoord
from astropy.table import Table, join
from astropy.io import ascii

# check that version of mastcasjobs is new enough
# we are using some features not in version 0.0.1
from pkg_resources import get_distribution
from packaging.version import Version as V

assert V(get_distribution("mastcasjobs").version) >= V('0.0.2'), """
A newer version of mastcasjobs is required.
Update mastcasjobs to current version using this command:
pip install --upgrade git+git://github.com/rlwastro/mastcasjobs@master
"""

import mastcasjobs

# set width for pprint
astropy.conf.max_width = 150

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import hsc catalog search utilites and image search utilites from local py files
from search_utils import *
from image_utils import *

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

In [ ]:
HSCContext = "HSCv3"

Set up Casjobs environment.

In [ ]:
import getpass
if not os.environ.get('CASJOBS_USERID'):
    os.environ['CASJOBS_USERID'] = input('Enter Casjobs UserID:')
if not os.environ.get('CASJOBS_PW'):
    os.environ['CASJOBS_PW'] = getpass.getpass('Enter Casjobs password:')

## <span style="color:red;">Step 1</span>

Create a catalog of objects to be cross-matched with the HSC. In this case, we want a catalog of known supernovae in 2005. This list was created using data from the [IAU Central Bureau for Astronomical Telegrams][1]. The [list][2] is in CSV format, and looks like this:

```lang-none
RA,Dec,Name,Host Galaxy
37.68021,-2.93883,SN 2005A,NGC 958
268.70342,71.54292,SN 2005B, UGC 11066
168.87258,60.75153,SN 2005C, Anon.
```

There are 367 objects listed in the file, which are located all over the sky.

NOTE: The coordinates for SN 2005c in the attached list have been changed from the IAU position to the improved position 202.4699167 47.17658333 (see discussion in Maund et al 2005, and Li et al. 2006).


  [1]: http://www.cbat.eps.harvard.edu/lists/Supernovae.html
  [2]: https://archive.stsci.edu/hst/hsc/help/use_case_4_v1/sn2005.csv

## <span style="color:red;">Step 2</span> - Go to the [MAST Discovery Portal][1]

Upload the catalog by clicking on the ![import][2] Upload File icon (<span style="color:blue;">blue</span>). Use the browse button to find the supernova file you downloaded in Step 1, then click on the import button. The catalog now has its own tab (<span style="color:green;">green</span>), and the AstroView window shows the location of the first object in the catalog.

![crossmatch_1][3]

  [1]: http://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
  [2]: screenshots/import.png
  [3]: screenshots/crossmatch_1.png

In [ ]:
# load the file from step 1,  and load the non-empty columns
url = 'https://archive.stsci.edu/hst/hsc/help/use_case_4_v1/sn2005.csv'
df = pd.read_csv(url, usecols=[0, 1, 2, 3]).rename({'RA': 'ra', 'Dec': 'dec', 'Name': 'target'}, axis=1)
df.to_csv('sn2005.csv', index=False)
df = pd.read_csv('sn2005.csv')
df.head()

In [ ]:
# specify the first row as the target
target = df.loc[0, 'Host Galaxy']
coord = SkyCoord.from_name(target)

ra = coord.ra.degree
dec = coord.dec.degree
radius = 500 # arcsec

print(f'ra: {ra}\ndec: {dec}')

In [ ]:
filters = "gri"
size_deg = 0.2
zoom = 0.33

pixel_size = 0.25 # PS1 pixel size in arcsec
# get size in pixels from image size in degrees
size = int(size_deg*3600/pixel_size)

In [ ]:
# color JPEG image
cim = getcolorim(ra, dec, zoom=zoom, size=size, filters=filters, format="jpg", autoscale=99.5)
print(f"Got {cim.size} color jpeg image")

# get Gaia EDR3 catalog (search a circle that just touches the image edges)
gcat = getgaia(ra, dec, radius=size_deg/2)
print(f"Got {len(gcat)} Gaia EDR3 sources")

# extract image WCS
iwcs = getwcs(cim)
gcat['x'], gcat['y'] = iwcs.wcs_world2pix(gcat['ra'], gcat['dec'], 0)

In [ ]:
target_xy = gcat[(gcat['ra'].round(11) == ra) & (gcat['dec'].round(11) == dec)]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_title(f'{target} PS1 {filters}')
# note we must specify the extent to get the pixel origin in the lower left corner
ax.imshow(cim, origin="upper", extent=(0, cim.size[0], 0, cim.size[1]))
ax.plot('x', 'y', 'o', markerfacecolor='none', data=gcat)
ax.plot('x', 'y', '+r', data=target_xy)
plt.show()

## <span style="color:red;">Step 3</span> - Perform the cross-matching

Click on the ![crossmatch_24][1] icon (<span style="color:blue;">blue</span>), which generates a popup where you can select the catalog (i.e. HSC; <span style="color:green;">green</span>) and matching radius (default of 3 arcsec; <span style="color:orange;">orange</span>); see below for a discussion of the search radius. Click Cross-Match (<span style="color:yellow;">yellow</span>).

![crossmatch_2][3]


  [1]: screenshots/crossmatch_24.png
  [2]: https://archive.stsci.edu/hst/hsc/help/use_case_4_v1.html#radius
  [3]: screenshots/crossmatch_2.png

### Cone search with HSC API

In [ ]:
radius = 1 # degrees
cone = hcvcone(ra, dec, radius, table="summary")
tab_hcv = ascii.read(cone)
df_hcv = tab_hcv.to_pandas()
df_hcv.sort_values('MatchID', inplace=True, ignore_index=True)

In [ ]:
df_hcv.head()

In [ ]:
len(df_hcv)

In [ ]:
# save typing a quoted list of columns

In [ ]:
columns = """MatchID,MatchRA,MatchDec,TargetName,NumFilters,NumVisits,NumImages,StartMJD,StopMJD""".split(",")

tab_hsc = hsccone(ra, dec, 1, table="summary", release='v3', columns=columns, verbose=True, format="table")
df_hsc = tab_hsc.to_pandas()

In [ ]:
df_hsc.head()

### Crossmatch

- [Catalogs.MAST - Hubble Source Catalog][1]
- [`POST /hsc/crossmatch/upload.*/?`][2]


  [1]: https://catalogs.mast.stsci.edu/docs/hsc.html#hubble-source-catalog
  [2]: https://catalogs.mast.stsci.edu/docs/hsc.html#post--hsc-crossmatch.*-?

In [ ]:
filename = 'sn2005.csv'
itab = Table.read(filename,format='ascii.csv')
print(f"Input table {filename} has {len(itab)} rows")

In [ ]:
itab

In [ ]:
url = 'https://catalogs.mast.stsci.edu/api/v0.1/hsc/v3/crossmatch/upload.csv'
r = 0.000833            # radius in degrees (3 arcsec)
r = requests.post(url, params=dict(radius=r), files=dict(file=open(filename, 'rb')))
rtab = Table.read(r.text,format='ascii.csv')
print(f"Match table has {len(rtab)} rows")

In [ ]:
rtab[:5]

In [ ]:
r.url

In [ ]:
tab = join(itab, rtab, keys_left=['ra','dec'], keys_right=['_ra_','_dec_'])
print(f"Joined table has {len(tab)} rows")
assert len(tab) == len(rtab), "Unexpected mismatch in length"
# delete the unnecessary _ra_ and _dec_ columns
del tab['_ra_']
del tab['_dec_']

cm = tab.to_pandas()
cm.head()

## <span style="color:red;">Step 4</span> - Review the results

There are 60 matches (<span style="color:blue;">blue</span>), although there can be multiple HSC matches for any given supernova if there are several HSC objects within the 3 arcsec of the supernova position (e.g. in this case, there are two matches for SN 2005H, two for SN2005P, 10 for SN 2005V). Note that the RA, Dec, Name, and Host Galaxy columns from the input catalog (<span style="color:green;">green</span>) are at the beginning of the Cross-Match table, followed by information (such as the distance between the catalog position and the HSC position (<span style="color:orange;">orange</span>)) from the HSC.

![crossmatch_3][1]

If you scroll over in the table, you will see the Target Name from the HST observation. For example, lines 3 and 4 (<span style="color:blue;">blue</span>) were given the name SN2005P, while lines 5-10 (<span style="color:green;">green</span>) were given the name of the galaxy. Note that this name is what the HST observer chose to call the object, so the name may not be useful in determining information about the target (e.g. the first target in the list is ambiguous).

![crossmatch_4][2]



  [1]: screenshots/crossmatch_3.png
  [2]: screenshots/crossmatch_4.png

In [ ]:
# create a dictionary of dataframes where the keys each TargetName and the values are a dataframe
targets = dict(tuple(cm.groupby('target')))

In [ ]:
targets.keys()

In [ ]:
targets['2005P']

In [ ]:
targets['2005V'].head()

## <span style="color:red;">Step 5</span> - Review the matches in AstroView

Under Filters on the left, the Name box shows you how many matches each supernova had with an HSC object (<span style="color:blue;">blue</span>); click on the "Show 16 more" button to see the complete list of the 21 (out of the 367) objects that had matches.

Clicking on the Focus button ![crosshair_24][1] icon (<span style="color:green;">green</span>) will center the object in the AstroView window (in this case SN 2005P). It's interesting to browse through some to see the locations of the supernovae in their galaxies. If you zoom the AstroView display, you can see both the HSC object(s) (in <span style="color:blue;">blue</span>) and the position of the supernova (in <span style="color:orange;">orange</span>) shown with different color symbols (<span style="color:yellow;">yellow</span>).

![crossmatch_5][2]



  [1]: screenshots/crosshair_24.png
  [2]: screenshots/crossmatch_5.png

In [ ]:
cm.value_counts('target')

## <span style="color:red;">Step 6</span> - Examine the HST data for SN 2005cs in detail

Click on the 2005cs button (<span style="color:blue;">blue</span>) to restrict the data to just this object. We find that there are 10 matches. If you scroll over to the right you will find that line 4 has NumImage = 9, while the others have NumImages between 1 and 5. Lets look at the detailed information for line 4 by clicking the Load Detailed Results ![detailed][1] icon on that row (<span style="color:green;">green</span>).

![crossmatch_10][2]

  [1]: screenshots/detailed.png
  [2]: screenshots/crossmatch_10.png

In [ ]:
# select data by target and sort by NumImages
target_2005cs = cm[cm.target.eq('2005cs')].copy().sort_values('NumImages', ascending=False, ignore_index=True)

In [ ]:
target_2005cs.info()

In [ ]:
target_2005cs.MatchID.nunique()

In [ ]:
# show top five rows and first 24 columns
target_2005cs.iloc[:5, :24]

This creates a new tab (<span style="color:blue;">blue</span>), and presents cutout preview images for each member of the match (<span style="color:green;">green</span>). Hover over the cutout and it will tell you the name of the image, in this case hst_12762_a6_wfc3_uvis_f689m. There are five measurements with WFPC2 and four WFC3 measurements (<span style="color:orange;">orange</span>) for this object, MatchID = 3972863. The WFC3 observations might be of particular interest since they were taken after the images discussed in the Maund et al. (2005) and Li et al. (2006) papers.

![crossmatch_11][3]

  [3]: screenshots/crossmatch_11.png

In [ ]:
# get Images for MatchID with the most NumImages and select data for WFC3
hsc_matches = ascii.read(hcvsearch(table='detailed', MatchID=target_2005cs.MatchID[0], Instrument='WFC3'))
hsc_matches

### Plot the brightest, median and faintest magnitudes

In [ ]:
# sort images by magnitude from brightest to faintest
phot = hsc_matches
isort = np.argsort(phot['MagAuto'])
# select the brightest, median and faintest magnitudes
ind = [isort[0], isort[len(isort)//2], isort[-1]]

# we plot zoomed-in and zoomed-out views side-by-side for each selected image
nim = len(ind)*2
ncols = 2 # images per row
nrows = (nim+ncols-1)//ncols

imsize1 = 19
imsize2 = 101
mra = hsc_matches['MatchRA'][0]
mdec = hsc_matches['MatchDec'][0]

# define figure and axes
fig, axes = plt.subplots(nrows, ncols, figsize=(12, (12/ncols)*nrows), tight_layout=True)

t0 = time.time()

# iterate through each set of two subplots in axes
for (ax1, ax2), k in zip(axes, ind):
    
    # get the images
    im1 = get_hla_cutout(phot['ImageName'][k], mra, mdec, size=imsize1)
    im2 = get_hla_cutout(phot['ImageName'][k], mra, mdec, size=imsize2)
    
    # plot left column   
    ax1.imshow(im1, origin="upper", cmap="gray")
    ax1.set_title(f"{phot['ImageName'][k]} m={phot['MagAuto'][k]:.3f}", fontsize=14)
    
    # plot right column
    ax2.imshow(im2, origin="upper", cmap="gray")
    xbox = np.array([-1, 1])*imsize1/2 + (imsize2-1)//2
    ax2.plot(xbox[[0, 1, 1, 0, 0]], xbox[[0, 0, 1, 1, 0]], 'r-', linewidth=1)
    ax2.set_title(f"{phot['ImageName'][k]} m={phot['MagAuto'][k]:.3f}", fontsize=14)    
    
print(f"{(time.time()-t0):.1f} s: got {nrows*ncols} cutouts")

## <span style="color:red;">Step 7</span> - Compare with Li et al. (2006)

Click on the Toggle Overlay Image ![overlay][1] icon (<span style="color:yellow;">yellow</span> in the image above) to overlay the Hubble image in AstroView. Another option is to click on the cutout preview image (<span style="color:green;">green</span>) itself to bring up the HLA Interactive Display, which provides more flexibility for viewing the image (e.g., changing the contrast level), although you will need to use the RA and DEC to find the object.

When you zoom in with AstroView, you can see the 10 potential matches (blue squares) to the original search position (orange square). The match that we selected is shown by the light blue square, and turns out to be very close to the position of SN2005cs, as shown by comparison with the Li et al. (2006) image below.

![crossmatch_12][2]

Note that the Li et al.(2006) paper (and Maund et al. (2005) primarily used ACS observations, which are NOT INCLUDED IN THE HSC. This is an important point, that while the HSC is a useful way to look for HST data, users should not assume there is no other data in the HLA if it is not listed in the HSC. In Step 8 we show how to make a more detailed evaluation of exisiting data using the HLA.



  [1]: screenshots/overlay.png
  [2]: screenshots/crossmatch_12.png

## <span style="color:red;">Step 8</span> - Go to the [HLA][1] to search for related data

Enter the coordinates (202.4699167 47.17658333) for SN2005cs into the search box (<span style="color:blue;">blue</span>); click on advanced search (<span style="color:green;">green</span>); then click on cutouts (<span style="color:orange;">orange</span>), and then search to see the Inventory. Click on the Images button (<span style="color:yellow;">yellow</span>) to view the cutouts.

![crossmatch_14][2]

Here are some of the cutouts, including HST_10498_01_ACS_HRC_F555W that has the supernova itself! This is around cutout #350 if you want to find it. It is not included in the HSC because it is an ACS/HRC image. Source lists for this detector will be added to the HSC in the future.

![crossmatch_15][3]

There are 824 images returned from the HLA search, though in many cases the "Cutout position is outside image". You can reduce the number to look at by going to the inventory view and selecting a subset (e.g., by putting *acs* in the box under Detector and clicking).


  [1]: http://hla.stsci.edu/
  [2]: screenshots/crossmatch_14.png
  [3]: screenshots/crossmatch_15.png

## <span style="color:red;">Search Radius selection</span>

The selection of the search size can be tricky, and depends on the science you are trying to accomplish. For this case, we want a radius large enough to find all the galaxies that have HST data, but small enough to not give you HSC objects that are not part of the galaxy. One approach is to start with a large search radius (say 20") and then try smaller ones until you start loosing cases you would like to keep. Note that a more typical search radius (e.g., for matching stellar fields) is ~0.2", which is roughly the accuracy of the absolute astrometry for the HSC.

Return to <span style="color:red;">Step 3</span>.

## Get list of HLA images for SN2005cs from `astroquery.mast`

Use the position of the HSC object with the largest `NumImages` for the search.

In [ ]:
mra = target_2005cs.ra[0]
mdec = target_2005cs.dec[0]
c = SkyCoord(mra, mdec, unit='deg')
obstab = Observations.query_criteria(coordinates=c, obs_collection='HLA', radius=0.0)
print(f"Found {len(obstab)} HLA observations")
obstab[obstab.colnames[:5]]

In [ ]:
# select observations for an example image
otab = obstab[np.char.startswith(np.char.lower(obstab['obs_id']),'hst_10498_01_acs_hrc')
              & (np.char.lower(obstab['filters'])!='detection')]
print(f"Found {len(otab)} observations for visit hst_10498_01_acs_hrc")
# sort from red to blue
otab = otab[np.argsort(0.5*(otab['em_min']+otab['em_max']))[::-1]]
otab['obs_id', 'em_min', 'em_max', 'filters']

In [ ]:
im = get_hla_cutout(otab['obs_id'][0], mra, mdec, size=256, autoscale=99.0,
                   instrument=otab['instrument_name'][0])

In [ ]:
plt.imshow(im, origin="upper", cmap="gray")

## Display a range of cutout images

This is chosen to approximately match the use case displayed images.

In [ ]:
otab = obstab[(np.char.lower(obstab['filters']) != 'detection')]
otab.sort('obs_id')
print(f"Found {len(otab)} observations skipping 'detection' filter")

# get index of the HRC observation
hindex = np.where(np.char.startswith(np.char.lower(otab['obs_id']),
                                     'hst_10498_01_acs_hrc_f555w'))[0][0]
print(f"Select index {hindex}")
# show a range around the selected image
otab['obs_id', 'instrument_name', 'filters'][hindex-4:hindex+5]

In [ ]:
plt.figure(1, (20, 20))
for i, j in enumerate(range(hindex-4, hindex+5), start=1):
    plt.subplot(3, 3, i)
    obs_id = otab['obs_id'][j]
    im = get_hla_cutout(obs_id, mra, mdec, size=256, autoscale=99.0,
                       instrument=otab['instrument_name'][j])
    print(f"Got image {j} {obs_id}")
    plt.imshow(im, origin="upper", cmap="gray")
    plt.title(obs_id)